In [2]:
!pip install -U notebook-as-pdf
pyppeteer-install

import pandas as pd
import numpy as np

Requirement already up-to-date: notebook-as-pdf in c:\programdata\anaconda3\lib\site-packages (0.5.0)


NameError: name 'pyppeteer' is not defined

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Question 1 Import a 311 NYC service request.

In [ ]:
df_customer_service =  pd.read_csv(r'C:\Users\user\jupyter notebook\Project_1_customer_support\Data-Science-with-Python-Project-2--master\311_Service_Requests_from_2010_to_Present.csv')

In [ ]:
df_customer_service.head()


## Question 2:- Read or convert the columns ‘Created Date’ and Closed Date’ to datetime datatype and create a new column ‘Request_Closing_Time’ as the time elapsed between request creation and request closing. (Hint: Explore the package/module datetime)

In [ ]:
df_customer_service['Created Date']=pd.to_datetime(df_customer_service['Created Date'])
df_customer_service['Closed date']=pd.to_datetime(df_customer_service['Closed Date'])
df_customer_service['Request_Closing_Time']=df_customer_service['Closed date']-df_customer_service['Created Date']

In [ ]:
df_customer_service.head()

## Question 3 :- Provide major insights/patterns that you can offer in a visual format (graphs or tables); at least 4 major conclusions that you can come up with after generic data mining.

In [ ]:
#insight - 1

df_customer_service.groupby('Complaint Type')['Location Type'].value_counts().to_frame()

In [ ]:
#insight - 2
df_customer_service.groupby('Agency')['Complaint Type'].value_counts().to_frame()

In [ ]:
#insight - 3
df_customer_service.groupby(['Agency','Descriptor'])['Complaint Type'].count().to_frame().sort_values('Complaint Type',ascending=False).head(5)

In [ ]:
#insight - 4
df_customer_service.groupby(['City','Descriptor'])['Complaint Type'].value_counts().to_frame().rename(columns={'Complaint Type':'Count'}).sort_values('Count',ascending=False)

In [ ]:
#insight - 5
df_customer_service.groupby('City')['Complaint Type'].count().to_frame().sort_values('Complaint Type', ascending=False).head(1)

## Question 4 :- Order the complaint types based on the average ‘Request_Closing_Time’, grouping them for different locations.

In [ ]:

#converting time closed  to minutes
df_customer_service['Request_Closing_Time_Minutes']=df_customer_service['Request_Closing_Time'].dt.seconds/60



#groupby using aggregated minutes took to close complaint
df_customer_service.groupby(['City', 'Complaint Type'])['Request_Closing_Time_Minutes'].mean().to_frame()
df_customer_service.head(2)

## Question 5 :-  For the below statements you need to state the Null and Alternate and then provide a statistical test to accept or reject the Null Hypothesis along with the corresponding ‘p-value’.

In [ ]:
#Question 5.1: Whether the average response time across complaint types is similar or not (overall)

df_customer_service['Request_Closing_Time_in_seconds'] = df_customer_service['Request_Closing_Time'].apply(lambda x : x.seconds)

from scipy.stats import ttest_ind 
average_response_time = {}
unique_complaint = set(df_customer_service['Complaint Type'].unique())

for i in unique_complaint:
    average_response_time[i] = df_customer_service[df_customer_service['Complaint Type'] == i]['Request_Closing_Time_in_seconds'].mean()
    

In [ ]:
#Question 5.2: Are the type of complaint or service requested and location related?

for i in unique_complaint:
    for k in set(unique_complaint) - {i,i}:
        print('mean for {} is : {}'.format(i , average_response_time[i]))        
        print('mean for {} is : {}'.format(k , average_response_time[k]))
        
        sat1 = list(df_customer_service[df_customer_service['Complaint Type'] == i]['Request_Closing_Time_in_seconds'].dropna())
        sat2 = list(df_customer_service[df_customer_service['Complaint Type'] == k]['Request_Closing_Time_in_seconds'].dropna())
        
        test_statistic , p_value = ttest_ind(sat1 , sat2)     
        print('{} --> p value for similar average for {} , {}\n\n'.format(round(p_value,3) , i , k))